In [1]:
import os, sys
import numpy as np
import pandas as pd
import skimage.io
from skimage.transform import resize

import PIL
from PIL import Image
import cv2
import warnings
import h5py
warnings.filterwarnings("ignore")
SIZE = 512

Using TensorFlow backend.


In [2]:
# Load dataset info
DIR = '../input/'
data = pd.read_csv('../input/train.csv')

In [3]:
def getTrainDataset():
    
    path_to_train = DIR + '/train/'
    data = pd.read_csv(DIR + '/train.csv')

    paths = []
    labels = []
    
    for name, lbl in zip(data['Id'], data['Target'].str.split(' ')):
        y = np.zeros(28)
        for key in lbl:
            y[int(key)] = 1
        paths.append(os.path.join(path_to_train, name))
        labels.append(y)

    return np.array(paths), np.array(labels)

def getTestDataset():
    
    path_to_test = DIR + '/test/'
    data = pd.read_csv(DIR + '/sample_submission.csv')

    paths = []
    labels = []
    
    for name in data['Id']:
        y = np.ones(28)
        paths.append(os.path.join(path_to_test, name))
        labels.append(y)

    return np.array(paths), np.array(labels)
paths, labels = getTrainDataset()

In [4]:
labels[0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [5]:
from tqdm import tqdm
hdf5_path = 'D:\Human-Protein-Atlas-Image-Classification\input\proteins.h5'

In [6]:
data = h5py.File(hdf5_path, "w")
data.create_dataset("photos", shape = (0, 512, 512, 4), maxshape = (None, 512, 512, 4)
                    #, chunks = True, compression = "gzip", compression_opts = 9
                   )

<HDF5 dataset "photos": shape (0, 512, 512, 4), type "<f4">

In [7]:
batch_size = 1024
x_max = 512
y_max = 512

In [8]:
channels = ["red", "blue", "green", "yellow"]
def resize_img(path):
    images = []
    for channel in channels:
        im = np.array(Image.open(path + '_' + channel + '.png'))
        images.append(im)
    im = np.stack((
                images
            ), -1)/255
    return im

In [9]:
import concurrent.futures
def data_gen(paths):
    for i in tqdm(range(0, len(paths), batch_size)):
        xs = []
        with concurrent.futures.ThreadPoolExecutor(max_workers = 16) as executor:
                for value in executor.map(resize_img, paths[i:i+batch_size]):
                    xs.append(value)
        final_images = np.array(xs)
        print(final_images.shape)
        data["photos"].resize(data["photos"].len() + final_images.shape[0], axis = 0)
        data["photos"][i:i+final_images.shape[0], :, :, :] = final_images
data_gen(paths)
data.close()

  0%|                                                                                           | 0/31 [00:00<?, ?it/s]

(1024, 512, 512, 4)


  3%|██▋                                                                                | 1/31 [00:20<10:29, 20.99s/it]

(1024, 512, 512, 4)


  6%|█████▎                                                                             | 2/31 [00:44<10:50, 22.44s/it]

...

 94%|████████████████████████████████████████████████████████████████████████████▋     | 29/31 [11:46<00:48, 24.38s/it]

(1024, 512, 512, 4)


 97%|███████████████████████████████████████████████████████████████████████████████▎  | 30/31 [12:12<00:24, 24.43s/it]

(352, 512, 512, 4)


100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [12:25<00:00, 24.04s/it]


In [10]:
data = h5py.File(hdf5_path, "r")

In [11]:
data["photos"][0]

array([[[0.03137255, 0.        , 0.03529412, 0.        ],
        [0.        , 0.        , 0.02352941, 0.        ],
        [0.        , 0.08235294, 0.10588235, 0.        ],
        ...,
        [0.        , 0.03529412, 0.00392157, 0.01568628],
        [0.        , 0.        , 0.        , 0.02352941],
        [0.        , 0.        , 0.        , 0.        ]],

       [[0.01960784, 0.01960784, 0.01176471, 0.07450981],
        [0.        , 0.02745098, 0.07843138, 0.03921569],
        [0.        , 0.03529412, 0.07843138, 0.        ],
        ...,
        [0.03529412, 0.        , 0.        , 0.00784314],
        [0.0627451 , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.01960784]],

       [[0.        , 0.04705882, 0.20392157, 0.        ],
        [0.        , 0.02745098, 0.05882353, 0.        ],
        [0.        , 0.05490196, 0.14117648, 0.        ],
        ...,
        [0.01176471, 0.        , 0.06666667, 0.01176471],
        [0.        , 0.      

In [12]:
%%time
for i in paths[:1000]:
    resize_img(i)

Wall time: 26.4 s


In [14]:
%%time
data["photos"][:1000]

Wall time: 14.2 s


array([[[[0.03137255, 0.        , 0.03529412, 0.        ],
         [0.        , 0.        , 0.02352941, 0.        ],
         [0.        , 0.08235294, 0.10588235, 0.        ],
         ...,
         [0.        , 0.03529412, 0.00392157, 0.01568628],
         [0.        , 0.        , 0.        , 0.02352941],
         [0.        , 0.        , 0.        , 0.        ]],

        [[0.01960784, 0.01960784, 0.01176471, 0.07450981],
         [0.        , 0.02745098, 0.07843138, 0.03921569],
         [0.        , 0.03529412, 0.07843138, 0.        ],
         ...,
         [0.03529412, 0.        , 0.        , 0.00784314],
         [0.0627451 , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.01960784]],

        [[0.        , 0.04705882, 0.20392157, 0.        ],
         [0.        , 0.02745098, 0.05882353, 0.        ],
         [0.        , 0.05490196, 0.14117648, 0.        ],
         ...,
         [0.01176471, 0.        , 0.06666667, 0.01176471],
         [